## Imports

In [7]:
import torch
import pydpf
import bootstrap_model
import pathlib

## Set options

In [8]:
dx = 25
dy = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data_path = pathlib.Path('.').parent.absolute().joinpath('data.csv') 

C:\Users\k24113585\PycharmProjects\dpf-baselining\jss_examples\Linear Gaussian Example\data.csv


## Generate Data

In [9]:
SSM = bootstrap_model.make_SSM(dx, dy, device)
pydpf.simulate_and_save(data_path, SSM=SSM, time_extent=1000, n_trajectories=2000, batch_size=100, device=device)

Warning - file already exists at C:\Users\k24113585\PycharmProjects\dpf-baselining\jss_examples\Linear Gaussian Example\data.csv, continuing could overwrite its data
Done                  



## Create Dataset

In [11]:
dataset = pydpf.StateSpaceDataset(data_path=data_path, 
                        series_id_column='series_id', 
                        state_prefix='state', 
                        observation_prefix='observation', 
                        device=device)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=100, shuffle=True, collate_fn=dataset.collate)

## Define a particle filter with multinomial resampling

In [20]:
multinomial_resampler = pydpf.MultinomialResampler(torch.Generator(device=device))
PF = pydpf.ParticleFilter(resampler=multinomial_resampler, SSM=SSM)

TypeError: MultinomialResampler.__init__() missing 1 required positional argument: 'generator'

## Run particle Filter

In [19]:
aggregation_function = pydpf.FilteringMean()
for state, observation in data_loader:
    estimated_state = PF(observation=observation, n_particles=1000, aggregation_function=aggregation_function, time_extent=1000)

tensor([[-6.8770e-01,  1.0152e+00,  3.9009e-01,  ...,  1.3003e-01,
          3.6376e-02,  3.2040e-02],
        [-1.1558e+00,  5.7689e-01,  1.4709e+00,  ...,  3.4972e-01,
          4.3954e-01,  1.9440e-01],
        [-6.5087e-01,  5.2488e-01,  4.4651e-01,  ...,  3.0194e-01,
          4.7781e-04,  5.9771e-01],
        ...,
        [ 3.3527e+04,  5.2536e+04,  7.0856e+04,  ...,  7.0857e+04,
          5.2538e+04,  3.3531e+04],
        [ 3.3921e+04,  5.3152e+04,  7.1687e+04,  ...,  7.1688e+04,
          5.3154e+04,  3.3923e+04],
        [ 3.4320e+04,  5.3774e+04,  7.2526e+04,  ...,  7.2529e+04,
          5.3777e+04,  3.4320e+04]], device='cuda:0')
tensor([[-6.9703e-01,  5.8370e-01,  2.7945e-01,  ...,  1.6620e-01,
          3.5275e-01, -8.1333e-02],
        [-2.2453e-01, -1.5871e-01,  7.7245e-01,  ...,  2.8684e-01,
         -5.1766e-01,  3.1352e-01],
        [ 5.5671e-01, -1.4454e-01, -3.4183e-01,  ...,  5.8264e-02,
         -1.9633e-01, -5.8449e-02],
        ...,
        [-6.7516e+02, -1.0572

KeyboardInterrupt: 